In [1]:
from dotenv import load_dotenv
load_dotenv()
import rich

In [2]:
from langfuse import Langfuse
langfuse = Langfuse()

### Create Langfuse prompt template

In [ ]:
langfuse.create_prompt(
    name="event-planner-chat",
    prompt=[
        {"role":"system", "content":"You are an senior event planner."},
        {"role":"user", "content": 
            "Plan an event titled {{Event_Name}}. The event will be about: {{Event_Description}}. "
            "The event will be held in {{Location}} on {{Date}}. "
            "Consider the following factors: audience, budget, venue, catering options, and entertainment. "
            "Provide a detailed plan including potential vendors and logistics.",
        }
    ],
    type="chat", # need to set type to chat, when creating a "chat" prompt(with role setting)
    config={
        "model":"gpt-4o-mini",
        "temperature": 0,
    },
    labels=["production"]
)

### Get Langfuse Prompt
* We can still write `fallback prompt` in the codebase, just in case. 
* `cache_ttl_seconds` is setting prompt cache in seconds. If hit the cache, then no fetch

In [9]:
langfuse_chat_prompt_template = langfuse.get_prompt(
    # name="event-planner",
    name="event-planner-chat",
    label="production",
    cache_ttl_seconds=30, # default is 60
    # default prompt if no cache and fetch is failed
    fallback="I'm sorry, I'm not sure what you're asking for. Could you please provide more details?",
    max_retries= 2 # default is 2(s), max is 4(s), max delay is 10(s)
)

We can get the prompt, config, version ... so on, from the object

In [10]:
rich.print(langfuse_chat_prompt_template.prompt)
rich.print(langfuse_chat_prompt_template.config)
print(langfuse_chat_prompt_template.version)

[
    {'role': 'system', 'content': 'You are an senior event planner.'},
    {
        'role': 'user',
        'content': 'Plan an event titled {{Event_Name}}. The event will be about: {{Event_Description}}. The event 
will be held in {{Location}} on {{Date}}. Consider the following factors: audience, budget, venue, catering 
options, and entertainment. Provide a detailed plan including potential vendors and logistics.'
    }
]

{'model': 'gpt-4o-mini', 'temperature': 0}

1


#### Check are we using fallback prompt
* We can set a warning, tag or trace, something for use to know fallback happened 

In [ ]:
print(langfuse_chat_prompt_template.is_fallback)

False


In [ ]:
langfuse_chat_prompt_template.compile(
    Event_Name="My Birthday Party",
    Event_Description="A fun party",
    Location="My house",
    Date="Saturday"
    )

[{'content': 'You are an senior event planner.', 'role': 'system'},
 {'content': 'Plan an event titled My Birthday Party. The event will be about: A fun party. The event will be held in My house on Saturday. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.',
  'role': 'user'}]

In [11]:
langchain_prompt_template = langfuse_chat_prompt_template.get_langchain_prompt()
print(langchain_prompt_template)
print(type(langchain_prompt_template))

[('system', 'You are an senior event planner.'), ('user', 'Plan an event titled {Event_Name}. The event will be about: {Event_Description}. The event will be held in {Location} on {Date}. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.')]
<class 'list'>


In [6]:
from langchain_core.prompts import ChatPromptTemplate

### Convert into langchain prompt template

In [12]:
langchain_prompt_template = ChatPromptTemplate.from_messages(
    messages=langfuse_chat_prompt_template.get_langchain_prompt(),
    )

# langchain_prompt_template = ChatPromptTemplate.from_messages(
#     messages=langfuse_chat_prompt_template.get_langchain_prompt(),
#     ).with_config(run_name="event planner pomrpt template")

#### Partial
* We can set format instruction by partial variable, so it will dynamically change with schema

In [24]:
print(langchain_prompt_template)
print(langchain_prompt_template.partial(Event_Name="My Birthday Party"))

input_variables=['Date', 'Event_Description', 'Event_Name', 'Location'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an senior event planner.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['Date', 'Event_Description', 'Event_Name', 'Location'], input_types={}, partial_variables={}, template='Plan an event titled {Event_Name}. The event will be about: {Event_Description}. The event will be held in {Location} on {Date}. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.'), additional_kwargs={})]
input_variables=['Date', 'Event_Description', 'Location'] input_types={} partial_variables={'Event_Name': 'My Birthday Party'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types=

### For tracking the prompt generation in Langfuse
* set the `langfuse_prompt` object in template metadata for Langfuse to track the gneration

In [13]:
langchain_prompt_template.metadata = {"langfuse_prompt": langfuse_chat_prompt_template}

In [10]:
rich.print(langchain_prompt_template)

ChatPromptTemplate(
    input_variables=['Date', 'Event_Description', 'Event_Name', 'Location'],
    input_types={},
    partial_variables={},
    metadata={'langfuse_prompt': <langfuse.model.ChatPromptClient object at 0x105fd1fd0>},
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                input_types={},
                partial_variables={},
                template='You are an senior event planner.'
            ),
            additional_kwargs={}
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['Date', 'Event_Description', 'Event_Name', 'Location'],
                input_types={},
                partial_variables={},
                template='Plan an event titled {Event_Name}. The event will be about: {Event_Description}. The 
event will be held in {Location} on {Date}. Consider the following factors: audience, budget, venue, catering 
options, and entertainment. Provide a detailed plan including potential vendors and logistics.'
            ),
            additional_kwargs={}
        )
    ]
)

In [14]:
model = langfuse_chat_prompt_template.config["model"]
temperature = str(langfuse_chat_prompt_template.config["temperature"])
print(f"Prompt model configurations\nModel: {model}\nTemperature: {temperature}")

Prompt model configurations
Model: gpt-4o-mini
Temperature: 0


In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=model, temperature=temperature)
chain = langchain_prompt_template | llm

In [16]:
example_input = {
    "Event_Name": "Wedding",
    "Event_Description": "The wedding of Julia and Alex, a charming couple who share a love for art and nature. This special day will celebrate their journey together with a blend of traditional and contemporary elements, reflecting their unique personalities.",
    "Location": "Central Park, New York City",
    "Date": "June 5, 2024"
}

### Set the callbackhandler for track the text generation

In [30]:
from langfuse.callback import CallbackHandler
langfuse_callback_handler = CallbackHandler(
    # if not specified here, it's in .env
    trace_name="planner_01",
    tags= ["event-planner", "sean", "good boy"]
)

#### Tests the SDK connection with the server

In [19]:
langfuse_callback_handler.auth_check()

True

`run_name` will passed to trace the prompt usage

In [31]:
# we pass the callback handler to the chain to trace the run in Langfuse
response = chain.invoke(input=example_input,
                        config={
                            "callbacks":[langfuse_callback_handler],
                            # "run_name": "test_123",
                            # "tags": ["event-planner", "sean", "good boy"]
                            }
                            )
# response = chain.invoke(input=example_input)

print(response.content)

### Event Plan: Wedding of Julia and Alex

**Event Title:** Wedding of Julia and Alex  
**Date:** June 5, 2024  
**Location:** Central Park, New York City  
**Theme:** A blend of traditional and contemporary elements celebrating art and nature.

---

### 1. Audience
- **Guests:** Approximately 150-200 close family and friends.
- **Demographics:** A mix of young adults, families, and older relatives, with a focus on art and nature enthusiasts.

### 2. Budget
- **Estimated Total Budget:** $50,000
  - Venue Permit: $2,000
  - Catering: $15,000
  - Decor: $10,000
  - Entertainment: $5,000
  - Photography/Videography: $3,000
  - Attire: $5,000
  - Invitations: $1,500
  - Miscellaneous (transportation, favors, etc.): $8,500

### 3. Venue
- **Location:** Central Park, specifically the Conservatory Garden or the Great Lawn.
- **Permit:** Apply for a wedding permit through the NYC Parks Department (approximately $2,000).
- **Logistics:** 
  - Set up a tent for inclement weather.
  - Arrange for